# Set-up Enviromnent

In [0]:
# setup sent2vec
! git clone https://github.com/epfml/sent2vec.git
%cd sent2vec
! make 

In [0]:
pip install .

In [1]:
!pip install langid

     |████████████████████████████████| 1.9MB 9.5MB/s 
  Created wheel for langid: filename=langid-1.1.6-cp36-none-any.whl size=1941190 sha256=28006626c5e62f25431f8c729bdc9d21c62bebae2c427dccf2ff73056cc49c18
  Stored in directory: /root/.cache/pip/wheels/29/bc/61/50a93be85d1afe9436c3dc61f38da8ad7b637a38af4824e86e
Successfully built langid


In [0]:
import pandas as pd
import sent2vec
from scipy import spatial
import nltk
from itertools import  chain
from itertools import repeat
import numpy as np
from sklearn.manifold import TSNE
import re
import langid
from sklearn.metrics import jaccard_score
from itertools import combinations
from itertools import product
from scipy import spatial
from statistics import mean

In [0]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
%cd ../ 

/content


In [0]:
!git clone https://github.com/LucaBassanese/Tesi

Cloning into 'Tesi'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 176 (delta 19), reused 27 (delta 10), pack-reused 129
Receiving objects: 100% (176/176), 304.41 MiB | 11.42 MiB/s, done.
Resolving deltas: 100% (49/49), done.
Checking out files: 100% (80/80), done.


In [0]:
%cd sent2vec

/content/sent2vec


# Creazione Embdedding

In [0]:
frasi = pd.read_csv('/content/Tesi/data/frasi.txt', header = None, names= ['parole'])
keyword = pd.read_csv('/content/Tesi/data/keywordvero.txt', header = None, names= ['parole']) #sbagliate, trovare Keyword più adatte
abstract = pd.read_excel('/content/Tesi/data/abstract.xlsx')

In [0]:
#seleziono solo dati utilizzabili dell'abstract
abstract = abstract.drop_duplicates(['Titolo'])
lingua = [langid.classify(testo)[0] for testo in abstract.Titolo]
lingua = pd.Series(lingua)
abstract = abstract[(lingua == 'en').values]

In [0]:
#creo dt
autore = abstract['autore: Nome'] + ' ' + abstract['autore: Cognome']
parole_chiavi = abstract['Parole chiave inglese']
dt = pd.concat([parole_chiavi, autore], axis= 1)
dt.columns = ['parole_chiave', 'autore']
dt =dt.dropna(subset= ['parole_chiave'])

In [0]:
#pulizia parole chiavi
chiavi = list(dt.parole_chiave)
chiavi = [re.sub(' \(all\)|\\n|-|\.', '', chi) for chi in chiavi]
chiavi = [chi.lower() for chi in chiavi]
chiavi_fin = [re.split('; |, ', chi) for chi in chiavi]
chiavi_fin = [[a.strip() for a in chi] for chi in chiavi_fin]

In [0]:
#creazione dataset autore parole chiavi per ogni singola parola chiave
lunghezza = [len(chi) for chi in chiavi_fin]
a = list(dt.autore)
auto = []
for i in range(len(lunghezza)):
  auto.append([a[i]] * lunghezza[i])
auto = list(chain.from_iterable(auto))
chiavi_fin = list(chain.from_iterable(chiavi_fin))
dizio = {'Autore': auto, 'Parole_chiave': chiavi_fin}
dataset = pd.DataFrame(dizio)

In [0]:
! ./fasttext sent2vec -input /content/Tesi/data/frasi.txt -output my_model -minCount 0 -dim 300 -epoch 20 -loss hs -wordNgrams 3

Read 0M words
Number of words:  3249
Number of labels: 0
tcmalloc: large alloc 2403901440 bytes == 0x55e042a78000 @  0x7f7fd0c4e887 0x55e038ecfeec 0x55e038eec378 0x55e038ee9c0b 0x55e038eee075 0x55e038eb6692 0x7f7fcfae3b97 0x55e038eb699a
Progress: 100.0%  words/sec/thread: 114818  lr: 0.000000  loss: 7.809718  eta: 0h0m 


In [0]:
model = sent2vec.Sent2vecModel()
model.load_model('my_model.bin') 

In [0]:
#creazione dataset con posizione keyword
key_emb = model.embed_sentences( list(dataset.Parole_chiave))
boh = pd.DataFrame(key_emb)
X_embed = TSNE(n_components=2).fit_transform(boh.values)
X_embed = pd.DataFrame(X_embed)
parziale_tsne = pd.concat([dataset, X_embed], axis=1)

In [0]:
parziale_tsne.columns = ['Autore', 'Parole_chiave', 'Tsne_1', 'Tsne_2']

In [0]:
b = parziale_tsne.groupby(['Autore', 'Parole_chiave'], as_index=False).size()

In [0]:
 a = parziale_tsne.groupby(['Autore', 'Parole_chiave'], as_index=False).count()

In [0]:
grouped = parziale_tsne.groupby(['Autore', 'Parole_chiave'], as_index=False)

In [0]:
semifin = grouped.agg({'Tsne_1': 'mean',
             'Tsne_2': 'mean'})

In [0]:
semifin['Conta'] = b.values

In [0]:
semifin.to_excel('auto_parole.xlsx', index=False)

# Jaccard similarity

In [0]:
output_series = parziale_tsne.groupby(['Autore'])['Parole_chiave'].apply(list)

In [0]:
output_series

Autore
AGNESE MARIA DI BRISCO    [poset, gender inequality, gender gap, hasse d...
AHMAD KABIR NAIMZADA      [environment, bifurcation, chaos, duopoly game...
ALDO SOLARI               [combining dependent permutation tests, multip...
ALESSANDRO SANTORO        [marginal tax reforms, marginal tax reforms, p...
ALICE ALBONICO            [inflation bias, limited asset market particip...
                                                ...                        
TANIA MASI                [state capacity, fiscal capacity, resource cur...
VALERIA CASTOLDI          [insight problem solving, subliminal hints, co...
VALERIA GATTAI            [intangible assets, ownership, whollyowned sub...
VALERIO VARINI            [business history, company town, enterprise, w...
VITTORIA CERASI           [banking, market structure and oligopoly, dere...
Name: Parole_chiave, Length: 77, dtype: object

In [0]:
def get_jaccard_sim(list1, list2): 
    a = set(list1) 
    b = set(list2)
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [0]:
def countX(lst, x): 
    count = 0
    for ele in lst: 
        if (ele == x): 
            count = count + 1
    return count 
countX(b, 'ciao')

0

In [0]:
def get_weighted_Jaccard(list1, list2):
  c = list(set(list1).union(set(list2)))
  conta1 = [countX(list1, unione) for unione in c]
  conta2  = [countX(list2, unione) for unione in c]
  return sum(np.minimum(conta1, conta2)) / sum(np.maximum(conta1, conta2))

In [0]:
from itertools import combinations
from itertools import product

In [0]:
professori = list(output_series.index)

In [0]:
professori

In [0]:
Confronto = []
for i in product(professori, professori):
  Confronto.append(i[0] + ' / ' + i[1])

In [0]:
# Con prodotto cartesiano
professori = list(output_series.index)
prod = product(professori, professori)
Jaccard = []
Jaccard_weighted = []
Autore1 = []
Autore2 =[]
for i in list(prod):
  Autore1.append(i[0])
  Autore2.append(i[1])
  Jaccard.append(get_jaccard_sim(output_series[i[0]], output_series[i[1]]))
  Jaccard_weighted.append(get_weighted_Jaccard(output_series[i[0]], output_series[i[1]]))

In [0]:
# Con combinazioni semplici
# professori = list(output_series.index)
# comb = combinations(professori, 2)
# Jaccard = []
# Jaccard_weighted = []
# Confronto = []
# for i in list(comb):
#   Confronto.append(i[0] + ' / ' + i[1])
#   Jaccard.append(get_jaccard_sim(output_series[i[0]], output_series[i[1]]))
#   Jaccard_weighted.append(get_weighted_Jaccard(output_series[i[0]], output_series[i[1]]))


In [0]:
dizionario = {'Autore1' : Autore1, 'Autore2' : Autore2, 'Similarità di Jaccard': Jaccard, 'Similarità di Jaccard pesata': Jaccard_weighted}

In [0]:
similarita_jaccards = pd.DataFrame(dizionario)

In [0]:
similarita_jaccards.to_excel('similarita_jaccards_prod.xlsx', index= False)

In [0]:
similarita_jaccards

# Titoli embedding

In [0]:
import pandas as pd
from scipy import spatial
import nltk
from itertools import  chain
from itertools import repeat
import numpy as np
from sklearn.manifold import TSNE
import re
import langid
from sklearn.metrics import jaccard_score
from itertools import combinations
from itertools import product
from scipy import spatial
from statistics import mean

In [4]:
!git clone https://github.com/LucaBassanese/Tesi

Cloning into 'Tesi'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 176 (delta 19), reused 27 (delta 10), pack-reused 129
Receiving objects: 100% (176/176), 304.41 MiB | 29.38 MiB/s, done.
Resolving deltas: 100% (49/49), done.
Checking out files: 100% (80/80), done.


In [0]:
dft = pd.read_excel('/content/Tesi/data/abstract.xlsx')
dft = dft.drop_duplicates(['Titolo'])
# Seleziono sogli i titoli in inglese
lingua = [langid.classify(testo)[0] for testo in dft.Titolo]
lingua = pd.Series(lingua)
dft = dft[(lingua == 'en').values]

In [0]:
frasi = pd.read_csv('/content/Tesi/data/frasi.txt', header=None)

In [0]:
autore = dft['autore: Nome'] + ' ' + dft['autore: Cognome']
dft['Autore'] = autore

## Sent2vec

In [0]:
embedding = pd.read_csv('/content/Tesi/data/sent2vec_tri_hs_300.csv')
X_embed = TSNE(n_components=2).fit_transform(embedding.values)
X_embed = pd.DataFrame(X_embed)
X_embed.index = dft.index
X_embed.columns = ['Tsne_1_sent2vec', 'Tsne_2_sent2vec']
totale = pd.concat([X_embed, dft], axis=1)

In [0]:
#totale.to_excel('titoli_embedding.xlsx')
#doc2vecunidbow500

## Doc2vec

In [0]:
embedding = pd.read_csv('/content/Tesi/data/doc2vec_uni_dbow_500.csv')
X_embed = TSNE(n_components=2).fit_transform(embedding.values)
X_embed = pd.DataFrame(X_embed)
X_embed.index = dft.index
X_embed.columns = ['Tsne_1_doc2vec', 'Tsne_2_doc2vec']

In [0]:
totale = pd.concat([X_embed, totale], axis=1)

## Global auto encode

In [0]:
embedding = pd.read_csv('/content/Tesi/data/universal.csv')
X_embed = TSNE(n_components=2).fit_transform(embedding.values)
X_embed = pd.DataFrame(X_embed)
X_embed.index = dft.index
X_embed.columns = ['Tsne_1_universal', 'Tsne_2_universal']
totale = pd.concat([X_embed, totale], axis=1)

## SBERT

In [0]:
embedding = pd.read_csv('/content/Tesi/data/SBERT.csv')
X_embed = TSNE(n_components=2).fit_transform(embedding.values)
X_embed = pd.DataFrame(X_embed)
X_embed.index = dft.index
X_embed.columns = ['Tsne_1_SBERT', 'Tsne_2_SBERT']
totale = pd.concat([X_embed, totale], axis=1)

In [0]:
totale.to_excel('titoli_embedding.xlsx', index= False)

In [0]:
totalino = totale[['Autore', 'Titolo', 'Abstract', 'Anno di pubblicazione',
 'Tsne_1_SBERT',
 'Tsne_2_SBERT',
 'Tsne_1_universal',
 'Tsne_2_universal',
 'Tsne_1_doc2vec',
 'Tsne_2_doc2vec',
 'Tsne_1_sent2vec',
 'Tsne_2_sent2vec', 'Abstract inglese', 'Parole chiave inglese', 'Parole chiave' ]]

In [0]:
totalino.to_excel('totalino.xlsx', index= False)

## Tutto in una colonna

In [0]:
#Sent2vec
embedding = pd.read_csv('/content/Tesi/data/sent2vec_tri_hs_300.csv')
X_embed = TSNE(n_components=2).fit_transform(embedding.values)
X_embed = pd.DataFrame(X_embed)
X_embed.index = dft.index
X_embed.columns = ['Tsne_1', 'Tsne_2']
data_sent = pd.concat([X_embed, dft], axis=1)

In [0]:
#doc2vec
embedding = pd.read_csv('/content/Tesi/data/doc2vec_uni_dbow_500.csv')
X_embed = TSNE(n_components=2).fit_transform(embedding.values)
X_embed = pd.DataFrame(X_embed)
X_embed.index = dft.index
X_embed.columns = ['Tsne_1', 'Tsne_2']
data_doc = pd.concat([X_embed, dft], axis=1)

In [0]:
#universal
embedding = pd.read_csv('/content/Tesi/data/universal.csv')
X_embed = TSNE(n_components=2).fit_transform(embedding.values)
X_embed = pd.DataFrame(X_embed)
X_embed.index = dft.index
X_embed.columns = ['Tsne_1', 'Tsne_2']
data_univ = pd.concat([X_embed, dft], axis=1)

In [0]:
#SBERT
embedding = pd.read_csv('/content/Tesi/data/SBERT.csv')
X_embed = TSNE(n_components=2).fit_transform(embedding.values)
X_embed = pd.DataFrame(X_embed)
X_embed.index = dft.index
X_embed.columns = ['Tsne_1', 'Tsne_2']
data_SBERT = pd.concat([X_embed, dft], axis=1)

In [0]:
totale_col =pd.concat([data_sent, data_doc, data_univ, data_SBERT], axis= 0)

In [0]:
totalino_col = totale_col[['Autore', 'Titolo', 'Abstract', 'Anno di pubblicazione',
 'Tsne_1',
 'Tsne_2',
  'Abstract inglese', 'Parole chiave inglese', 'Parole chiave' ]]

In [0]:
Modello = ['Sent2vec'] * 1548 + ['Doc2vec'] * 1548 + ['Universal'] * 1548 + ['SBERT'] * 1548

In [0]:
totalino_col['Modello'] = Modello


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
totalino_col.to_excel('totalino_col.xlsx', index=False)

# Cosine similarity

In [0]:
#carico embedding di sent2vec, runnare prima titoli embedding
embedding = pd.read_csv('/content/Tesi/data/sent2vec_tri_hs_300.csv')
embedding.index = dft.index
dft_sent = pd.concat([embedding, dft], axis=1)

In [0]:
#definisco la funzione per la cosine similarity
def get_cosine_similarity_titoli(str1, str2):
  ind1 = list(dft[(dft_sent.Autore == str1)].index)
  ind2 = list(dft[(dft_sent.Autore == str2)].index)
  result = list(product(ind1, ind2))
  similarity = [spatial.distance.cosine(embedding.loc[re[0]], embedding.loc[re[1]]) for re in result]
  return mean(similarity)

In [0]:
# professori = list(dft['Autore'].drop_duplicates()) 
# comb = combinations(professori, 2)
# cosine_similarity = []
# Confronto = []
# for i in list(comb):
#   Confronto.append(i[0] + ' / ' + i[1])
#   cosine_similarity.append(get_cosine_similarity_titoli(i[0], i[1]))

In [0]:
%%time
professori = list(dft['Autore'].drop_duplicates()) 
prod = product(professori, professori)
cosine_similarity = []
Autore1 = []
Autore2 = []
for i in list(prod):
  Autore1.append(i[0])
  Autore2.append(i[1])
  cosine_similarity.append(get_cosine_similarity_titoli(i[0], i[1]))

CPU times: user 15min 13s, sys: 867 ms, total: 15min 14s
Wall time: 15min 13s


In [0]:
dizionario = {'Autore1': Autore1, 'Autore2' : Autore2, 'Cosine similarity': cosine_similarity}
ciao = pd.DataFrame(dizionario)

In [0]:
ciao

In [0]:
ciao.to_excel('Relazione_titoli_fin.xlsx', index=False)

## Doc2vec

In [0]:
#carico embedding di doc2vec
embedding = pd.read_csv('/content/Tesi/data/doc2vec_uni_dbow_500.csv')
embedding.index = dft.index
dft_sent = pd.concat([embedding, dft], axis=1)

In [33]:
%%time
professori = list(dft['Autore'].drop_duplicates()) 
prod = product(professori, professori)
cosine_similarity = []
Autore1 = []
Autore2 = []
for i in list(prod):
  Autore1.append(i[0])
  Autore2.append(i[1])
  cosine_similarity.append(get_cosine_similarity_titoli(i[0], i[1]))

CPU times: user 14min 7s, sys: 477 ms, total: 14min 7s
Wall time: 14min 7s


In [0]:
dizionario = {'Autore1': Autore1, 'Autore2' : Autore2, 'Cosine similarity': cosine_similarity}
ciao = pd.DataFrame(dizionario)

In [0]:
ciao

In [36]:
ciao['Cosine similarity']

0       0.540331
1       0.644599
2       0.655629
3       0.688888
4       0.651680
          ...   
6079    0.725311
6080    0.526922
6081    0.683425
6082    0.627705
6083    0.569923
Name: Cosine similarity, Length: 6084, dtype: float64

In [0]:
ciao.to_excel('Relazione_titoli_doc2vec.xlsx', index=False)

In [0]:
# professori = list(dft['Autore'].drop_duplicates()) 
# comb = combinations(professori, 2)
# cosine_similarity = []
# Confronto = []
# for i in list(comb):
#   Confronto.append(i[0] + ' / ' + i[1])
#   cosine_similarity.append(get_cosine_similarity_titoli(i[0], i[1]))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


In [0]:
# dizionario = {'Autori a confronto': Confronto, 'Cosine similarity': cosine_similarity}
# ciao = pd.DataFrame(dizionario)

In [0]:
# ciao.to_excel('Relazione_titoli_doc2vec.xlsx', index=False)

## Universal

In [0]:
#carico embedding di universal
embedding = pd.read_csv('/content/Tesi/data/universal.csv')
embedding.index = dft.index
dft_sent = pd.concat([embedding, dft], axis=1)

In [22]:
%%time
professori = list(dft['Autore'].drop_duplicates()) 
prod = product(professori, professori)
cosine_similarity = []
Autore1 = []
Autore2 = []
for i in list(prod):
  Autore1.append(i[0])
  Autore2.append(i[1])
  cosine_similarity.append(get_cosine_similarity_titoli(i[0], i[1]))

CPU times: user 14min 15s, sys: 333 ms, total: 14min 16s
Wall time: 14min 16s


In [0]:
dizionario = {'Autore1': Autore1, 'Autore2' : Autore2, 'Cosine similarity': cosine_similarity}
ciao = pd.DataFrame(dizionario)

In [0]:
ciao

In [0]:
ciao.to_excel('Relazione_titoli_universal.xlsx', index=False)

In [0]:
# %%time
# professori = list(dft['Autore'].drop_duplicates()) 
# comb = combinations(professori, 2)
# cosine_similarity = []
# Confronto = []
# for i in list(comb):
#   Confronto.append(i[0] + ' / ' + i[1])
#   cosine_similarity.append(get_cosine_similarity_titoli(i[0], i[1]))

CPU times: user 7min 25s, sys: 138 ms, total: 7min 26s
Wall time: 7min 26s


In [0]:
# dizionario = {'Autori a confronto': Confronto, 'Cosine similarity': cosine_similarity}
# ciao = pd.DataFrame(dizionario)

In [0]:
# ciao.to_excel('Relazione_titoli_universal.xlsx', index=False)

## SBERT

In [0]:
#carico embedding di SBERT
embedding = pd.read_csv('/content/Tesi/data/SBERT.csv')
embedding.index = dft.index
dft_sent = pd.concat([embedding, dft], axis=1)

In [28]:
%%time
professori = list(dft['Autore'].drop_duplicates()) 
prod = product(professori, professori)
cosine_similarity = []
Autore1 = []
Autore2 = []
for i in list(prod):
  Autore1.append(i[0])
  Autore2.append(i[1])
  cosine_similarity.append(get_cosine_similarity_titoli(i[0], i[1]))

CPU times: user 14min 9s, sys: 168 ms, total: 14min 9s
Wall time: 14min 9s


In [0]:
dizionario = {'Autore1': Autore1, 'Autore2' : Autore2, 'Cosine similarity': cosine_similarity}
ciao = pd.DataFrame(dizionario)

In [31]:
ciao

,Autore1,Autore2,Cosine similarity
0,AHMAD KABIR NAIMZADA,AHMAD KABIR NAIMZADA,0.444766
1,AHMAD KABIR NAIMZADA,LUCA MATTEO STANCA,0.549571
2,AHMAD KABIR NAIMZADA,MATTEO MANERA,0.531136
3,AHMAD KABIR NAIMZADA,MARIAPIA MENDOLA,0.598216
4,AHMAD KABIR NAIMZADA,ANDREA ONGARO,0.540336
...,...,...,...
6079,PAOLO BERTOLETTI,ROBERTO ASCARI,0.478284
6080,PAOLO BERTOLETTI,RICCARDO CORRADIN,0.440860
6081,PAOLO BERTOLETTI,MENGESHA YAYO NEGASI,0.633847
6082,PAOLO BERTOLETTI,BIANCA BARBARO,0.450583


In [0]:
ciao.to_excel('Relazione_titoli_SBERT.xlsx', index=False)

In [0]:
# %%time
# professori = list(dft['Autore'].drop_duplicates()) 
# comb = combinations(professori, 2)
# cosine_similarity = []
# Confronto = []
# for i in list(comb):
#   Confronto.append(i[0] + ' / ' + i[1])
#   cosine_similarity.append(get_cosine_similarity_titoli(i[0], i[1]))

CPU times: user 7min 25s, sys: 130 ms, total: 7min 25s
Wall time: 7min 25s


In [0]:
# dizionario = {'Autori a confronto': Confronto, 'Cosine similarity': cosine_similarity}
# ciao = pd.DataFrame(dizionario)

In [0]:
# ciao

In [0]:
ciao.to_excel('Relazione_titoli_SBERT.xlsx', index=False)

# Altre cose

In [0]:

#from itertools import chain
#separati = list(chain.from_iterable(separati))
separati2 = [chiave.split('; ') for chiave in separati ]

In [0]:
iniziale = abstract['Parole chiave inglese'].dropna()

In [0]:
chiavi = list(iniziale)
separati = [chiave.split(', ') for chiave in chiavi ]
from itertools import chain
separati = list(chain.from_iterable(separati))
separati2 = [chiave.split('; ') for chiave in separati ]
chiavifinal = list(chain.from_iterable(separati2))
chiavifinal = [parola.lower() for parola in chiavifinal]

In [0]:
chiavifinal = pd.Series(chiavifinal)

In [0]:
chiavifinal.drop_duplicates

In [0]:
! ./fasttext sent2vec -input frasi.txt -output my_model -minCount 0 -dim 300 -epoch 20 -loss hs -wordNgrams 3

Read 0M words
Number of words:  3249
Number of labels: 0
tcmalloc: large alloc 2403901440 bytes == 0x55bfa5b3c000 @  0x7f248158f887 0x55bf9d893eec 0x55bf9d8b0378 0x55bf9d8adc0b 0x55bf9d8b2075 0x55bf9d87a692 0x7f2480424b97 0x55bf9d87a99a
Progress: 100.0%  words/sec/thread: 102980  lr: 0.000000  loss: 7.788723  eta: 0h0m 


In [0]:
model = sent2vec.Sent2vecModel()
model.load_model('my_model.bin') 

In [0]:
key_emb = model.embed_sentences( list(dataset.parole))

# Vecchia Parole e Keyword

In [0]:
uni_embs, vocab = model.get_unigram_embeddings()

In [0]:
stop_words = set(stopwords.words('english')) 
vocab_pul = [w for w in vocab if not w in stop_words] 

In [0]:
a = pd.DataFrame(uni_embs)

In [0]:
b = pd.Series(vocab)

In [0]:
c = pd.Series(vocab_pul)

In [0]:
buleano = b.isin(c)

In [0]:
uni_embs = uni_embs[buleano.values]

In [0]:
key_emb = model.embed_sentences( list(keyword.parole))

In [0]:
similarity = []
for key in key_emb:
  sotto = []
  for paro in uni_embs:
    sotto.append(round(1-spatial.distance.cosine(key, paro), 3))
  similarity.append(sotto)


/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [0]:
indici = [sorted(range(len(sim)), key=lambda i: sim[i])[-20:] for sim in similarity]

In [0]:
len(similarity[0])

3163

In [0]:
parole = [[vocab_pul[i] for i in ind] for ind in indici]


In [0]:
parole_fin = list(chain.from_iterable(parole))

In [0]:
emb = [[uni_embs[i] for i in ind] for ind in indici]

In [0]:
vettori = []
for boh in emb:
  for ciao in boh:
    vettori.append(ciao)

In [0]:
finale = pd.DataFrame(vettori)

In [0]:
finale.columns

RangeIndex(start=0, stop=300, step=1)

In [0]:
finale['Parole'] = parole_fin

In [0]:
associata = np.repeat(list(keyword.parole), 20)

In [0]:
finale['keyword associata'] = associata

In [0]:
parziale = pd.DataFrame(key_emb)

In [0]:
parziale['Parole'] = keyword.parole

In [0]:
parziale['keyword associata'] = keyword.parole

In [0]:
finalissimo = pd.concat([finale, parziale], ignore_index= True)

In [0]:
finalissimo['Genere'] =  ['Parola'] * 600 + ['Keyword'] * 30

In [0]:
pertsne = finalissimo.iloc[:, :300].values

In [0]:
 X_embedded = TSNE(n_components=2).fit_transform(pertsne)

In [0]:
tsnedata = pd.DataFrame(X_embedded)

In [0]:
mappa = pd.concat([finalissimo, tsnedata], axis= 1)

In [0]:
basta = mappa.iloc[:, 300:]


In [0]:
basta.to_excel('Basta.xlsx', index=False)

In [0]:
import nltk

In [0]:
nltk.download('stopwords')
from nltk.corpus import stopwords

In [0]:
stop_words = set(stopwords.words('english')) 

In [0]:
vocab_pul = [w for w in vocab if not w in stop_words] 

3163

In [0]:
30*3249

97470

In [0]:
for a in uni_embs:
  print(len(a))

In [0]:
model = sent2vec.Sent2vecModel()
model.load_model('my_model.bin') 
comb = combinations(list(parole.parole), 2)
sims = []

for i in list(comb):
  v1 = model.embed_sentence(i[0])
  v2 = model.embed_sentence(i[1])
  sim = round(1-spatial.distance.cosine(v1, v2), 3)
  # print(f'First: {i[0]}, Second: {i[1]},\
  # Similarity: {sim}')
  sims.append(sim)